In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix,accuracy_score
import seaborn as sns
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.feature_selection import mutual_info_classif
import copy
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
import statistics
import plotly.graph_objs as go
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from keras._tf_keras.keras.models import Sequential
from keras._tf_keras.keras.layers import Dense
from keras._tf_keras.keras.optimizers import Adam
from keras._tf_keras.keras.losses import BinaryCrossentropy
from keras._tf_keras.keras import backend as K
import numpy as np
import tensorflow as tf
from keras._tf_keras.keras.optimizers import SGD, Adam
import random

/var/folders/ct/fjh_5m0n2zj9r5lpl1nrd20m0000gn/T/ipykernel_60676/2829960177.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
2024-05-18 15:27:27.203819: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
class CustomMLPClassifier:
    def __init__(self, hidden_layer_size = 100, max_iter=100, random_state=None, class_weight =None, input_dim = 23):
        if random_state:
            tf.random.set_seed(random_state)
            np.random.seed(random_state)
            random.seed(random_state)
        self.model = Sequential([
            Dense(hidden_layer_size, activation='relu', input_dim=input_dim),
            Dense(95, activation='relu'),
            Dense(95, activation='relu'),
            Dense(1, activation='sigmoid')
        ])
        max_iter = 40
        self.class_weight = class_weight
        self.max_iter = max_iter
        self.random_state = random_state

    def compile_model(self):
        optimizer=Adam(learning_rate=0.0002)
        loss='binary_crossentropy'
        if self.class_weight:
            binary_crossentropy_loss = BinaryCrossentropy()
            custom_loss_lambda = lambda y_true, y_pred: tf.where(tf.logical_and(y_true == 1, y_true != y_pred), 2800 * binary_crossentropy_loss(y_true, y_pred), 1*binary_crossentropy_loss(y_true, y_pred))
            self.model.compile(optimizer=optimizer, loss=custom_loss_lambda, metrics=['accuracy'], )
        else:
            self.model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

    def fit(self, X_train, y_train, **kwargs):
        X_train = X_train.values
        y_train = y_train.values
        self.model.fit(X_train, y_train, epochs=self.max_iter, batch_size=32, verbose=1, initial_epoch=1, validation_split=0.1, shuffle=False, **kwargs)

    def predict(self, X_test):
        dta = self.model.predict(X_test)
        arr_list = dta.flatten().tolist()
        grtr = 0
        lssr = 0
        arr_list_int = []
        for k in arr_list:
            if k > 0.5:
                arr_list_int.append(1)
                grtr = grtr + 1
            else:
                arr_list_int.append(0)
                lssr = lssr + 1

        return arr_list_int

In [6]:
class DataAnalyzer:
    def __init__(self, balance_weights = True, algo = "knn"):
        file_path = 'default of credit card clients.csv'

        self.dataset = pd.read_csv(file_path, header=None)
        self.dataset.columns = self.dataset.iloc[1]

        self.dataset = self.dataset.drop([0, 1])
        self.dataset = self.dataset.drop(self.dataset.columns[0], axis=1)
        # Reset the index
        self.dataset = self.dataset.reset_index(drop=True)
        self.original_dataset = self.dataset
        self.convert_columns()
        self.balance_weights = balance_weights
        self.algo = algo
        self.knn_weight = "uniform"
        self.class_weight = None
        if balance_weights:
            self.knn_weight = "distance"
            self.class_weight = "balanced"
        print(self.class_weight)

    
    def convert_columns(self):
        # Convert all columns to float except 'default payment next month' and 'Net Income Flag'
        for column in self.dataset.columns:
            if column not in ["ID", "SEX", "EDUCATION", "MARRIAGE", "PAY_0", "PAY_2", "PAY_3", "PAY_4", "PAY_5", "PAY_6", "default payment next month"]:
                self.dataset[column] = self.dataset[column].astype(float)
            else:
                self.dataset[column] = self.dataset[column].astype(int)

    def show_class_distribution(self):
        target_column = 'default payment next month'
        # Load your dataset into a pandas DataFrame

        # Step 2: Inspect the Target Variable
        class_distribution = self.dataset[target_column].value_counts()

        # Step 3: Visualize the Distribution
        class_distribution.plot(kind='bar', title='Target Variable Distribution')
        print(class_distribution)
        plt.xlabel('Class')
        plt.ylabel('Count')
        plt.show()

    def show_heat_map(self, dataset = None):
        if dataset is None:
            corr_matrix = self.dataset.corr()
        else:
            corr_matrix = dataset.corr()
        plt.figure(figsize=(20, 20))
        sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
        plt.title("Correlation Matrix")
        plt.show()

    def get_x_and_Y(self):
        self.X = self.dataset.drop('default payment next month', axis=1)  # Assuming 'p' is the target variable
        self.y = self.dataset['default payment next month']
        return (self.X, self.y)
    
    def perform_manual_splitting_without_cv(self):
        X_train, self.X_test_final, y_train, self.y_test_final = train_test_split(self.X, self.y, test_size=0.2, random_state=0)
        self.X_train, self.y_train = X_train, y_train
    
    def perform_manual_splitting_cv(self):
        # First split to get training set and first test set
        X_train, X_temp, y_train, y_temp = train_test_split(self.X, self.y, test_size=0.4, random_state=0)
        self.X_train, self.y_train = X_train, y_train
        self.X_cv, self.X_test_final, self.y_cv, self.y_test_final = train_test_split(X_temp, y_temp, test_size=0.5, random_state=0)
    
    def perform_kfold_cv(self, num_of_columns = None):
        X_train, X_test_final, y_train, y_test_final = train_test_split(self.X, self.y, test_size=0.2, random_state=0)
        if num_of_columns:
            X_train = copy.deepcopy(X_train.iloc[:, :num_of_columns])
            X_test_final = copy.deepcopy(X_test_final.iloc[:, :num_of_columns])
        self.X_train = X_train
        self.y_train = y_train.astype(int)
        self.X_test_final = X_test_final.reindex(X_train.columns, axis=1)
        self.y_test_final = y_test_final.astype(int)
        if self.algo == "knn":
            classifier = KNeighborsClassifier(n_neighbors = 40, metric = 'minkowski', p = 2.7, weights=self.knn_weight)
        elif self.algo == "rf":
            classifier = RandomForestClassifier(n_estimators=100, random_state=42, class_weight=self.class_weight)
        elif self.algo == "svm":
            classifier = SVC(kernel='rbf', random_state=42, class_weight=self.class_weight)
        elif self.algo == "logreg":
            classifier = LogisticRegression(random_state=42, class_weight=self.class_weight)
        elif self.algo == "nn":
            # classifier = MLPClassifier(hidden_layer_sizes=(100,), max_iter=100, random_state=42) 
            custom_mlp = CustomMLPClassifier(max_iter=100, random_state=42, class_weight=self.class_weight)
            custom_mlp.compile_model()  # You may need to pass additional parameters here depending on your requirements
            classifier = custom_mlp

        # Perform k-fold cross-validation
        kf = KFold(n_splits=5, shuffle=True, random_state=0)

        scores = []
        best_model = None
        best_avg_score = 0.0 

        for train_index, val_index in kf.split(X_train):
            X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
            y_train_fold, y_val_fold = y_train.iloc[train_index].astype(int), y_train.iloc[val_index].astype(int)

            # Train the model on the training fold
            X_train_smote, y_train_smote = X_train_fold, y_train_fold
            classifier.fit(X_train_smote, y_train_smote)

            # Evaluate the model on the validation fold
            y_val_pred = classifier.predict(X_val_fold)
            score = accuracy_score(y_val_fold, y_val_pred)

            # Update the best model if the current model has a better average performance
            if score > best_avg_score:
                best_avg_score = score
                best_model = classifier
            scores.append(score)
        
        self.classifier = best_model
        return(best_avg_score, scores)
    
    def perform_filter_methods(self):
        data = copy.deepcopy(self.dataset)
        X = copy.deepcopy(data).drop(columns=['default payment next month'])  # Features
        print("total features = ", len(X.columns))
        y = copy.deepcopy(data['default payment next month']).astype(int) 
        # Calculate Mutual Information scores
        mi_scores = mutual_info_classif(X, y)
        # Select features with MI score > 0.5
        selected_features = X.columns[mi_scores > 0.01]
        print("selected features with high mutual information with the target = ", len(selected_features))
        # Calculate correlation matrix
        corr_matrix = X[selected_features].corr().abs()
        copy_ds = copy.deepcopy(data[selected_features])
        # Remove one of two highly correlated features
        to_drop = set()
        dropped_columns = {}
        for i in range(len(corr_matrix.columns)):
            if i not in dropped_columns:
                for j in range(len(corr_matrix.columns)):
                    if abs(corr_matrix.iloc[i, j]) > 0.8 and i!=j and j not in dropped_columns and i not in dropped_columns and i!=j:
                        
                        colname_i = corr_matrix.columns[i]
                        colname_j = corr_matrix.columns[j]
                        # Calculate the Mutual Information score of each feature with the target variable
                        mi_i = mutual_info_classif(X[colname_i].values.reshape(-1, 1), y)[0]
                        mi_j = mutual_info_classif(X[colname_j].values.reshape(-1, 1), y)[0]
                        # Keep the feature with higher Mutual Information score
                        if mi_i > mi_j:
                            to_drop.add(colname_j)
                            dropped_columns[j] = colname_j
                        else:
                            to_drop.add(colname_i)
                            dropped_columns[i] = colname_i
        final_selected_features = selected_features.drop(to_drop)
        print("final selected features after removing one of two correlated features = ", len(final_selected_features))
        self.dataset = self.dataset[final_selected_features]
        self.dataset["default payment next month"] = data["default payment next month"]
        self.show_heat_map()
        return self.dataset
    
    def perform_wrapper_method(self):
        feature_scores = []
        feature_scores_dict = {}
        total_num_of_columns = len(self.X.columns)
        for i in range(1,total_num_of_columns+1):
            newAnlyzr = DataAnalyzer(balance_weights=self.balance_weights)
            newAnlyzr.get_x_and_Y()
            (best_avg_score, scores) = newAnlyzr.perform_kfold_cv(num_of_columns=i)
            mean_score, y_tst, y_prd = newAnlyzr.get_accuracy()
            feature_scores.append((i, mean_score))
            feature_scores_dict[i] = mean_score
        self.feature_scores = feature_scores
        additional_feature_penalty = -0.05
        score_weight = 0.95
        adjusted_scores = []
        best_num_of_features = 1
        best_adjusted_score = 0
        for (j, mn_score) in feature_scores:
            adjusted_score = mn_score*score_weight + j*additional_feature_penalty/total_num_of_columns
            adjusted_scores.append((j, adjusted_score))
            if adjusted_score > best_adjusted_score:
                best_adjusted_score = adjusted_score
                best_num_of_features = j
        best_score = feature_scores_dict[best_num_of_features]

        return {
            "best_num_of_features": best_num_of_features,
            "best_score": best_score
        }
    
    def plot_wrapper_scores(self):
        x_values, y_values = zip(*self.feature_scores)
        # Create a Plotly trace
        trace = go.Scatter(x=x_values, y=y_values, mode='lines+markers')

        # Create a Plotly layout
        layout = go.Layout(
            title='Wrapper Method Feature Scores',
            xaxis=dict(title='Number of Features'),
            yaxis=dict(title='Accuracy')
        )

        # Create a Plotly figure
        fig = go.Figure(data=[trace], layout=layout)

        # Display the plot
        fig.show()

    def perform_pca(self, n_components=None):
        # Standardize the data
        scaler = StandardScaler()
        data = self.X
        standardized_data = scaler.fit_transform(data)

        # Create PCA object
        pca = PCA(n_components=n_components)

        # Fit and transform the data
        pca_data = pca.fit_transform(standardized_data)

        # Create a DataFrame for the PCA results
        pca_columns = [f"PC{i+1}" for i in range(pca_data.shape[1])]
        pca_df = pd.DataFrame(data=pca_data, columns=pca_columns)

        # Concatenate with original dataset
        self.X = pca_df

    
    def perform_without_cv(self):
        if self.algo == "knn":
            classifier = KNeighborsClassifier(n_neighbors = 40, metric = 'minkowski', p = 2.7, weights=self.knn_weight)
        elif self.algo == "rf":
            classifier = RandomForestClassifier(n_estimators=100, random_state=42, class_weight=self.class_weight)
        elif self.algo == "svm":
            classifier = SVC(kernel='rbf', random_state=42, class_weight=self.class_weight)
        elif self.algo == "logreg":
            classifier = LogisticRegression(random_state=42, class_weight=self.class_weight)
        elif self.algo == "nn":
            # classifier = MLPClassifier(hidden_layer_sizes=(100,), max_iter=100, random_state=42,) 
            custom_mlp = CustomMLPClassifier(max_iter=100, random_state=42, class_weight=self.class_weight)
            custom_mlp.compile_model()  # You may need to pass additional parameters here depending on your requirements
            classifier = custom_mlp
        self.classifier = classifier
        self.classifier.fit(self.X_train, self.y_train)
        y_pred1 = self.classifier.predict(self.X_test_final)
        self.ac_final = accuracy_score(self.y_test_final,y_pred1)
        return self.ac_final, self.y_test_final
    
    def perform_knn(self):
        self.classifier = KNeighborsClassifier(n_neighbors = 20, metric = 'minkowski', p = 2.7, weights=self.knn_weight)
        self.classifier.fit(self.X_train, self.y_train)
        y_pred1 = self.classifier.predict(self.X_cv)
        self.ac_cv = accuracy_score(self.y_cv,y_pred1)
        return self.ac_cv
    
    def perform_random_forest(self):
        self.classifier = RandomForestClassifier(n_estimators=100, random_state=42, class_weight=self.class_weight)  # Initialize the Random Forest Classifier
        self.classifier.fit(self.X_train, self.y_train)  # Train the classifier
        y_pred = self.classifier.predict(self.X_cv)  # Perform prediction on the cross-validation set
        self.ac_cv = accuracy_score(self.y_cv, y_pred)  # Calculate accuracy
        return self.ac_cv
    
    def perform_svm(self):
        self.classifier = SVC(kernel='rbf', random_state=42, class_weight=self.class_weight)  # Initialize the SVM Classifier with RBF kernel
        self.classifier.fit(self.X_train, self.y_train)  # Train the classifier
        y_pred = self.classifier.predict(self.X_cv)  # Perform prediction on the cross-validation set
        self.ac_cv = accuracy_score(self.y_cv, y_pred)  # Calculate accuracy
        return self.ac_cv
    
    def perform_logistic_regression(self):
        self.classifier = LogisticRegression(random_state=42, class_weight=self.class_weight)  # Initialize the Logistic Regression Classifier
        self.classifier.fit(self.X_train, self.y_train)  # Train the classifier
        y_pred = self.classifier.predict(self.X_cv)  # Perform prediction on the cross-validation set
        self.ac_cv = accuracy_score(self.y_cv, y_pred)  # Calculate accuracy
        return self.ac_cv
    
    def perform_neural_network(self):
        # self.classifier = MLPClassifier(hidden_layer_sizes=(100,), max_iter=100, random_state=42)  # Initialize the MLP Classifier with one hidden layer of 100 neurons
        input_dim = len(self.X_train.values[0])
        custom_mlp = CustomMLPClassifier(max_iter=100, random_state=42, class_weight=self.class_weight, input_dim=input_dim)
        custom_mlp.compile_model()  # You may need to pass additional parameters here depending on your requirements
        self.classifier = custom_mlp
        self.classifier.fit(self.X_train, self.y_train)  # Train the classifier
        y_pred = self.classifier.predict(self.X_cv)  # Perform prediction on the cross-validation set
        self.ac_cv = accuracy_score(self.y_cv, y_pred)  # Calculate accuracy
        return self.ac_cv
    
    def get_accuracy(self):
        y_pred_final = self.classifier.predict(self.X_test_final)
        print("kkk", 1 in y_pred_final)
        print("lll", 0 in y_pred_final)
        self.ac_final = accuracy_score(self.y_test_final,y_pred_final)
        return self.ac_final, self.y_test_final, y_pred_final

class Master:
    def __init__(self, balance_weights = True, algo = "knn"):
        self.results = {}
        self.balance_weights = balance_weights
        self.algo = algo

    def print_classification_report(self, y_true, y_pred):
        report = classification_report(y_true, y_pred, target_names=['Negative Class', 'Positive Class'], output_dict=True)
        print("{:<20} {:<15} {:<15} {:<15} {:<15}".format('', 'precision', 'recall', 'f1-score', 'support'))
        for class_name, metrics in report.items():
            if class_name in ['accuracy', 'macro avg', 'weighted avg']:
                continue
            print("{:<20} {:<15.2f} {:<15.2f} {:<15.2f} {:<15}".format(class_name,
                                                                       metrics['precision'],
                                                                       metrics['recall'],
                                                                       metrics['f1-score'],
                                                                       metrics['support']))
        
    def organize_results(self):
        items = []
        for k, v in self.results.items():
            print(k, v)
            if isinstance(v, dict):
                for xk, xv in v.items():
                    new_x_key = xk
                    items.append((new_x_key, xv))
            else:
                new_key = k
                items.append((new_key, v))
        r_items = dict(items)
        df = pd.DataFrame({'method': list(r_items.keys()), 'result': list(r_items.values())})
        return df

    def run_without_cv(self):
        anlyzr = DataAnalyzer(balance_weights=self.balance_weights, algo=self.algo)
        anlyzr.get_x_and_Y()
        anlyzr.perform_manual_splitting_without_cv()
        rslts, y_pred = anlyzr.perform_without_cv()
        self.results["run without cv"] = rslts
        self.print_classification_report(anlyzr.y_test_final, y_pred)

    def run_with_cv(self):
        anlyzr = DataAnalyzer(balance_weights=self.balance_weights, algo=self.algo)
        anlyzr.get_x_and_Y()
        
        anlyzr.perform_manual_splitting_cv()
        if self.algo == "knn":
            anlyzr.perform_knn()
        elif self.algo == "rf":
            anlyzr.perform_random_forest()
        elif self.algo == "svm":
            anlyzr.perform_svm()
        elif self.algo == "logreg":
            anlyzr.perform_logistic_regression()
        elif self.algo == "nn":
            anlyzr.perform_neural_network()
        self.results["run with cv"], y_test_final, y_pred_final = anlyzr.get_accuracy()
        self.print_classification_report(y_test_final, y_pred_final)

    def run_with_kfold(self):
        anlyzr = DataAnalyzer(balance_weights=self.balance_weights, algo=self.algo)
        anlyzr.get_x_and_Y()
        (best_avg_score, scores) = anlyzr.perform_kfold_cv()
        rslt, y_test_final, y_pred_final =  anlyzr.get_accuracy()
        self.results["after_k_fold_run"] = {
            "best kfold cv score": best_avg_score,
            "scores": scores,
            "final kfold score on test dataset": rslt
        }
        self.print_classification_report(y_test_final, y_pred_final)

    def run_with_filter(self):
        # Implement feature selection using filter method (e.g., correlation)
        anlyzr = DataAnalyzer(balance_weights=self.balance_weights, algo=self.algo)
        anlyzr.perform_filter_methods()
        anlyzr.get_x_and_Y()
        anlyzr.perform_manual_splitting_cv()
        if self.algo == "knn":
            anlyzr.perform_knn()
        elif self.algo == "rf":
            anlyzr.perform_random_forest()
        elif self.algo == "svm":
            anlyzr.perform_svm()
        elif self.algo == "logreg":
            anlyzr.perform_logistic_regression()
        elif self.algo == "nn":
            anlyzr.perform_neural_network()
        self.results["with_filter_method"], y_test_final, y_pred_final  = anlyzr.get_accuracy()
        self.print_classification_report(y_test_final, y_pred_final)

    def run_with_wrapper(self):
        anlyzr = DataAnalyzer(balance_weights=self.balance_weights, algo=self.algo)
        anlyzr.get_x_and_Y()
        anlyzr.perform_manual_splitting_cv()
        self.results = anlyzr.perform_wrapper_method()
        anlyzr.plot_wrapper_scores()

    def run_with_pca(self):
        # Implement feature selection using PCA
        anlyzr = DataAnalyzer(balance_weights=self.balance_weights, algo=self.algo)
        anlyzr.get_x_and_Y()
        anlyzr.perform_pca(n_components=2)
        anlyzr.perform_manual_splitting_cv()
        if self.algo == "knn":
            anlyzr.perform_knn()
        elif self.algo == "rf":
            anlyzr.perform_random_forest()
        elif self.algo == "svm":
            anlyzr.perform_svm()
        elif self.algo == "logreg":
            anlyzr.perform_logistic_regression()
        elif self.algo == "nn":
            anlyzr.perform_neural_network()
        self.results["with_pca"], y_test_final, y_pred_final  = anlyzr.get_accuracy()
        self.print_classification_report(y_test_final, y_pred_final)
    
    def run_cv_problem(self):
        self.run_without_cv()
        self.run_with_cv()
        self.run_with_kfold()
        return self.results



In [7]:
filter_mstr = Master(balance_weights=True, algo="knn")
filter_mstr.run_with_pca()
filter_mstr.organize_results()

balanced
kkk True
lll True
                     precision       recall          f1-score        support        
Negative Class       0.82            0.94            0.87            4666.0         
Positive Class       0.56            0.29            0.38            1334.0         
with_pca 0.7918333333333333


,method,result
0,with_pca,0.791833


In [8]:
filter_mstr = Master(balance_weights=True, algo="rf")
filter_mstr.run_with_pca()
filter_mstr.organize_results()

balanced
kkk True
lll True
                     precision       recall          f1-score        support        
Negative Class       0.82            0.92            0.87            4666.0         
Positive Class       0.53            0.30            0.39            1334.0         
with_pca 0.786


,method,result
0,with_pca,0.786


In [9]:
filter_mstr = Master(balance_weights=True, algo="svm")
filter_mstr.run_with_pca()
filter_mstr.organize_results()

balanced
kkk True
lll True
                     precision       recall          f1-score        support        
Negative Class       0.86            0.85            0.86            4666.0         
Positive Class       0.50            0.50            0.50            1334.0         
with_pca 0.7755


,method,result
0,with_pca,0.7755


In [10]:
filter_mstr = Master(balance_weights=True, algo="logreg")
filter_mstr.run_with_pca()
filter_mstr.organize_results()

balanced
kkk True
lll True
                     precision       recall          f1-score        support        
Negative Class       0.86            0.64            0.73            4666.0         
Positive Class       0.33            0.64            0.44            1334.0         
with_pca 0.6368333333333334


,method,result
0,with_pca,0.636833


In [13]:
filter_mstr = Master(balance_weights=True, algo="nn")
filter_mstr.run_with_cv()
filter_mstr.organize_results()

balanced
Epoch 2/40


/Users/mirbilal/Desktop/ASSGNS_MLL/assgn_3/assgn_3_env/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


507/507 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.5977 - loss: 173291.4531 - val_accuracy: 0.3322 - val_loss: 57277.9453
Epoch 3/40
507/507 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5834 - loss: 78275.3281 - val_accuracy: 0.6272 - val_loss: 33029.3398
Epoch 4/40
507/507 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4388 - loss: 37145.3867 - val_accuracy: 0.7772 - val_loss: 159748.6406
Epoch 5/40
507/507 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5365 - loss: 51519.6914 - val_accuracy: 0.7644 - val_loss: 88531.9609
Epoch 6/40
507/507 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5654 - loss: 45043.7852 - val_accuracy: 0.6039 - val_loss: 53208.0000
Epoch 7/40
507/507 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5027 - loss: 29864.2129 - val_accuracy: 0.3711 - val_loss: 25284.5996
Epoch 8/40
507/507 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.3564 - loss: 15056.4131 - val_accuracy: 0.7683 - val_loss: 61556.0430
Epoch 9/40
507/507 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - acc

,method,result
0,run with cv,0.685333


In [11]:
filter_mstr = Master(balance_weights=True, algo="nn")
filter_mstr.run_with_pca()
filter_mstr.organize_results()

balanced
Epoch 2/40


/Users/mirbilal/Desktop/ASSGNS_MLL/assgn_3/assgn_3_env/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


507/507 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.7508 - loss: 362.0458 - val_accuracy: 0.8106 - val_loss: 314.1719
Epoch 3/40
507/507 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8035 - loss: 307.5367 - val_accuracy: 0.8111 - val_loss: 305.2805
Epoch 4/40
507/507 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8032 - loss: 302.1535 - val_accuracy: 0.8083 - val_loss: 303.3139
Epoch 5/40
507/507 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8036 - loss: 301.2557 - val_accuracy: 0.8089 - val_loss: 302.6328
Epoch 6/40
507/507 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8037 - loss: 300.8263 - val_accuracy: 0.8094 - val_loss: 302.4312
Epoch 7/40
507/507 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8041 - loss: 300.5564 - val_accuracy: 0.8078 - val_loss: 302.3050
Epoch 8/40
507/507 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8039 - loss: 300.3409 - val_accuracy: 0.8083 - val_loss: 302.2118
Epoch 9/40
507/507 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8038 - loss: 300.1708

,method,result
0,with_pca,0.808
